In [ ]:
import json
import time
import time
import pandas as pd
from pygments.lexers import q
import mysql.connector
from mysql.connector import Error
import re
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus.reader.tagged import word_tokenize
from nltk.corpus import stopwords
import numpy as np

In [ ]:
def create_connection(host_name, user_name, user_password, db_name, port):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            database=db_name,
            port=port
        )
        cursor = connection.cursor()
        # cursor.execute("SET GLOBAL max_allowed_packet=67108864")
        cursor.close()
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

In [3]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [9]:
host_name = "92.53.120.173"
user_name = "root"
user_password = "fd6_PejvxFF?@-"
db_name = "recsys_project_database"
port = 3306

In [16]:
connection = create_connection(host_name, user_name, user_password, db_name, port)

Connection to MySQL DB successful


In [11]:
cursor = connection.cursor()
try:
    cursor.execute("SELECT COUNT(*) FROM titles")
    row_count = cursor.fetchone()[0]
    print(f"Number of rows in 'titles' table: {row_count}")
except Error as e:
    print(f"The error '{e}' occurred")
finally:
    cursor.close()

Number of rows in 'titles' table: 386209


In [17]:
query = """
        SELECT t.*, d.description
        FROM titles t
        LEFT JOIN description_modified d ON t.tconst = d.tconst
        LIMIT 10;
        """
cursor = connection.cursor()
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

('tt0011801', 'movie', 'Tötet nicht mehr', 'In Russia s revolution, a violinist s son dies,the father is jailed for assaulting the governor. Years later, the violinist s daughter wants to marry the son of the man who sentenced her father.A tragic fate awaits her new husband.', 2019, None, 0, 'russia revolution violinist son diesthe father jailed assaulting governor year later violinist daughter want marry son man sentenced fathera tragic fate awaits new husband')
('tt0013274', 'movie', 'Istoriya grazhdanskoy voyny', 'A feature length documentary taken by the Red Army during the Russian Civil War, showing the incremental street battles and the major assaults waged against the counter revolutionaries.', 2021, None, 0, 'feature length documentary taken red army russian civil war showing incremental street battle major assault waged counter revolutionary')
('tt0062336', 'movie', 'The Tango of the Widower and Its Distorting Mirror', 'The story of a man whose wife has committed suicide and a

In [6]:
# def drop_all_tables(connection):
#     try:
#         cursor = connection.cursor()
#         cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")  # Отключить проверку внешних ключей
#         cursor.execute("SHOW TABLES;")
#         tables = cursor.fetchall()
#         for table in tables:
#             drop_query = f"DROP TABLE IF EXISTS {table[0]};"
#             print(f"Table {table[0]} dropped successfully")
#             cursor.execute(drop_query)
#         cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")  # Включить проверку внешних ключей обратно
#         connection.commit()
#         print("All tables dropped successfully")
#     except Error as e:
#         print(f"The error '{e}' occurred")
# drop_all_tables(connection)

In [7]:
# create_titles_table =  """
# SELECT COUNT(*) FROM titles
# """
# execute_query(connection, create_titles_table)

In [ ]:
create_titles_table =  """
    CREATE TABLE IF NOT EXISTS titles (
        tconst VARCHAR(11) PRIMARY KEY,
        title_type VARCHAR(20),
        title_name VARCHAR(255),
        description TEXT,
        year_start INT,
        year_end INT,
        is_adult TINYINT(1) NOT NULL DEFAULT 0
    );
    """
execute_query(connection, create_titles_table)

In [ ]:
# alter_query = """
#     ALTER TABLE titles
#     MODIFY COLUMN title_name VARCHAR(255);
#     """
# try:
#     cursor = connection.cursor()
#     cursor.execute(alter_query)
#     connection.commit()
#     print("Table altered successfully")
# except Error as e:
#     print(f"The error '{e}' occurred")

In [ ]:
create_genres_table = """
CREATE TABLE IF NOT EXISTS genres (
    genre_id TINYINT PRIMARY KEY,
    genre_name VARCHAR(20)
);
"""
execute_query(connection, create_genres_table)

In [ ]:
create_names_table = """
CREATE TABLE IF NOT EXISTS names (
    nconst VARCHAR(11) PRIMARY KEY,
    name VARCHAR(50),
    birth_year INT,
    death_year INT
);
"""
execute_query(connection, create_names_table)

In [ ]:
create_names_image_table = """
CREATE TABLE IF NOT EXISTS names_image (
    nconst VARCHAR(11),
    image_url TEXT,
    PRIMARY KEY (nconst),
    FOREIGN KEY (nconst) REFERENCES names (nconst)
);
"""
execute_query(connection, create_names_image_table)

In [ ]:
create_names_titles_table = """
CREATE TABLE IF NOT EXISTS names_titles (
    id INT AUTO_INCREMENT PRIMARY KEY,
    nconst VARCHAR(11),
    tconst VARCHAR(11),
    n_role ENUM('actor', 'director', 'producer'),
    character_played VARCHAR(255),
    actor_importance TINYINT,
    FOREIGN KEY (tconst) REFERENCES titles(tconst)
);
"""
# FOREIGN KEY (nconst) REFERENCES names(nconst),
execute_query(connection, create_names_titles_table)

In [ ]:
# cursor = connection.cursor()
# try:
#     cursor.execute("""
#             SELECT CONSTRAINT_NAME 
#             FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE 
#             WHERE TABLE_NAME = 'names_titles' 
#               AND COLUMN_NAME = 'nconst';
#             """)
#     foreign_key = cursor.fetchone()
# 
#     if foreign_key:
#         # Удаление внешнего ключа
#         drop_fk_query = f"ALTER TABLE names_titles DROP FOREIGN KEY {foreign_key[0]}"
#         cursor.execute(drop_fk_query)
#         connection.commit()
#         print("Foreign key for 'nconst' dropped successfully")
#     else:
#         print("No foreign key found for 'nconst'")
# except Error as e:
#     print(f"The error '{e}' occurred")
# finally:
#     cursor.close()

In [ ]:

create_ratings_table = """
CREATE TABLE IF NOT EXISTS ratings (
    tconst VARCHAR(11),
    rating FLOAT,
    num_votes INT,
    PRIMARY KEY (tconst),
    FOREIGN KEY (tconst) REFERENCES titles (tconst)
);
"""
execute_query(connection, create_ratings_table)

In [ ]:

create_titles_image_table = """
CREATE TABLE IF NOT EXISTS titles_image (
    tconst VARCHAR(11),
    image_url TEXT,
    PRIMARY KEY (tconst),
    FOREIGN KEY (tconst) REFERENCES titles (tconst)
);
"""
execute_query(connection, create_titles_image_table)

In [ ]:
create_titles_genres_table = """
CREATE TABLE IF NOT EXISTS titles_genres (
    id INT AUTO_INCREMENT PRIMARY KEY,
    tconst VARCHAR(11),
    genre_id TINYINT,
    FOREIGN KEY (tconst) REFERENCES titles (tconst),
    FOREIGN KEY (genre_id) REFERENCES genres (genre_id)
);
"""
execute_query(connection, create_titles_genres_table)

In [ ]:
create_user_profile_table = """
CREATE TABLE IF NOT EXISTS user_profile (
    user_id VARCHAR(20) PRIMARY KEY,
    name VARCHAR(30),
    age TINYINT,
    photo_uri VARCHAR(255)
);
"""
execute_query(connection, create_user_profile_table)

In [ ]:

create_user_collections_table = """
CREATE TABLE IF NOT EXISTS user_collections (
    collection_id INT AUTO_INCREMENT PRIMARY KEY,
    collection_name VARCHAR(30),
    user_id VARCHAR(20),
    FOREIGN KEY (user_id) REFERENCES user_profile (user_id)
);
"""
execute_query(connection, create_user_collections_table)

In [ ]:
create_titles_collections_table = """
CREATE TABLE IF NOT EXISTS titles_collections (
    id INT AUTO_INCREMENT PRIMARY KEY,
    collection_id INT,
    tconst VARCHAR(11),
    FOREIGN KEY (collection_id) REFERENCES user_collections (collection_id),
    FOREIGN KEY (tconst) REFERENCES titles (tconst)
);
"""
execute_query(connection, create_titles_collections_table)

In [ ]:

create_user_ratings_table = """
CREATE TABLE IF NOT EXISTS user_ratings (
    id INT AUTO_INCREMENT PRIMARY KEY,
    user_id VARCHAR(20),
    tconst VARCHAR(11),
    rating TINYINT,
    FOREIGN KEY (user_id) REFERENCES user_profile (user_id),
    FOREIGN KEY (tconst) REFERENCES titles (tconst)
);
"""
execute_query(connection, create_user_ratings_table)

In [ ]:
create_auth_table = """
CREATE TABLE IF NOT EXISTS auth (
    id INT AUTO_INCREMENT PRIMARY KEY,
    user_id VARCHAR(20),
    pass VARCHAR(16),
    FOREIGN KEY (user_id) REFERENCES user_profile (user_id)
);
"""
execute_query(connection, create_auth_table)

In [ ]:
add_default_collections_script = """
    CREATE TRIGGER after_user_insert
    AFTER INSERT ON user_profile
    FOR EACH ROW
    BEGIN
        INSERT INTO user_collections (collection_name, user_id)
        VALUES ('Watch later', NEW.user_id),
               ('Liked', NEW.user_id);
    END;
    """
execute_query(connection, add_default_collections_script)

In [ ]:
delete_user_script = """
    CREATE TRIGGER before_user_delete
    BEFORE DELETE ON user_profile
    FOR EACH ROW
    BEGIN
        DELETE FROM titles_collections WHERE collection_id IN (
            SELECT collection_id FROM user_collections WHERE user_id = OLD.user_id
        );
        DELETE FROM user_collections WHERE user_id = OLD.user_id;
        DELETE FROM user_ratings WHERE user_id = OLD.user_id;
        DELETE FROM auth WHERE user_id = OLD.user_id;
    END;
    """
execute_query(connection, delete_user_script)

In [ ]:

create_table_query = """
    CREATE TABLE IF NOT EXISTS description_modified (
        tconst VARCHAR(11) PRIMARY KEY,
        description TEXT,
        FOREIGN KEY (tconst) REFERENCES titles(tconst)
    );
    """
execute_query(connection, create_table_query)

Reading parsed_data

In [ ]:
with open("parsed_data.csv", "r", encoding="utf-8") as f:
    text = f.read()
columns = ['tconst', 'data']

In [ ]:
pattern = re.compile(r'}\s*\n\s*tt')
data = {}
matches = pattern.finditer(text)
indices = [match.start() for match in matches]
beginning = 0
for i in indices:
    a = text[beginning:i+1].split(';', 1)
    data[a[0]] = a[1]
    beginning = i + 2

In [ ]:
all_title_links = []
all_descriptions = []
for k, v in data.items():
    d = [k]
    link = re.findall(r'"image":"(https?://[^"]+)"', v)
    descriptions = re.findall(r'"description":"([^"]+)"', v)
    if link:
        d.append(link[0])
    else:
        d.append('https://mainstream-spb.ru/uploads/s/0/z/k/0zk8gjqzwgii/img/full_Yc96J5UA.png')
    if descriptions:
        description = re.sub(r"&[^;]+;", ' ', descriptions[0])
        description = re.sub(r'\s+', ' ', description)
        d.append(description)
    else:
        d.append('\\N')
    all_descriptions.append(d)

In [ ]:
desc_df = pd.DataFrame(all_descriptions, columns=['tconst', 'description', 'links'])
desc_df

In [ ]:
titlebasics = pd.read_csv('data/title.basics.tsv', sep = '\t')
titlebasics = pd.merge(titlebasics, desc_df, on='tconst', how='inner')
titlebasics = titlebasics.replace('\\N', None)
titlebasics

In [ ]:
titlebasics = titlebasics.sort_values(by='startYear', ascending=False)
# titlebasics = titlebasics[titlebasics['startYear'] <= 2023]
titlebasics_cleaned = titlebasics.dropna(subset=['startYear'])
titlebasics_cleaned['startYear'] = titlebasics_cleaned['startYear'].astype(int)
titlebasics_cleaned = titlebasics_cleaned[titlebasics_cleaned['startYear'] <= 2023]
titlebasics_cleaned

In [ ]:
title_principals = pd.read_csv('data/title.principals.tsv', sep = '\t')
title_principals

In [ ]:
title_principals_filtered = title_principals[title_principals['tconst'].isin(titlebasics_cleaned['tconst'])]
title_principals_filtered

In [ ]:
title_principals_filtered = title_principals_filtered.replace('\\N', None)
title_principals_filtered

In [ ]:
title_principals_filtered.head()

In [ ]:
title_principals_filtered['category'].unique()

In [ ]:
title_principals_filtered_rows = title_principals_filtered.to_numpy().tolist()

In [ ]:
name_set = []
names_titles_data = {}
for record in title_principals_filtered_rows:
    temp = []
    if record[0] not in names_titles_data:
        names_titles_data[record[0]] = []
    if record[3] in ['actress', 'actor']:
        if record[-1]:
            character = json.loads(record[-1])
            character = character[0].split('-')[0].strip()
        else:
            character = record[-1]
        temp = [record[2], record[0], 'actor', character, record[1]]
        name_set.append(record[2])
    elif record[3] in ['producer']:
        temp = [record[2], record[0], record[3], None, record[1]]
        name_set.append(record[2])
    elif record[3] in ['director']:
        temp = [record[2], record[0], record[3], None, None]
        name_set.append(record[2])
    if temp:
        names_titles_data[record[0]].append(temp)

In [ ]:
name_set = list(set(name_set))

In [ ]:
names_titles_data

In [ ]:
name_basics = pd.read_csv('data/name.basics.tsv', sep = '\t')
name_basics = name_basics.replace('\\N', None)
name_basics.head(20)

In [ ]:
name_basics = name_basics[name_basics['nconst'].isin(name_set)]
name_basics

In [ ]:
name_basics_list = name_basics.to_numpy().tolist()

In [ ]:
name_basics

In [ ]:
names_list = []
for name in name_basics_list:
    names_list.append([name[0], name[1], name[2], name[3]])

In [59]:
from itertools import islice

def close_connection(connection):
    if connection is not None and connection.is_connected():
        connection.close()
        print("Connection to MySQL DB closed")

def insert_names(connection, names):
    insert_query = """
    INSERT IGNORE INTO names (nconst, name, birth_year, death_year)
    VALUES (%s, %s, %s, %s)
    """
    cursor = connection.cursor()
    try:
        cursor.executemany(insert_query, names)
        connection.commit()
        print(f"{cursor.rowcount} rows inserted successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
    finally:
        cursor.close()

def chunked_iterator(data, chunk_size):
    it = iter(data)
    for first in it:
        yield list(islice(it, chunk_size - 1, None)) + [first]


chunks = enumerate(chunked_iterator(names_list, 100))


In [60]:
connection = create_connection(host_name, user_name, user_password, db_name, port)
if connection is not None:
    for i, chunk in chunks:
        insert_names(connection, chunk)
        if (i + 1) % 1000 == 0:  # Переподключение каждые 10000 записей
            print('reconnecting to db...')
            close_connection(connection)
            connection = create_connection(host_name, user_name, user_password, db_name, port)
            print('reconnected!')
    close_connection(connection)

Connection to MySQL DB successful
The error '2013 (HY000): Lost connection to MySQL server during query' occurred


In [ ]:
titlebasics_cleaned

In [ ]:
data_listed = titlebasics_cleaned.to_numpy().tolist() #!!!!!!


In [ ]:
g = titlebasics_cleaned['genres'].unique()
gg = []
ggg = [] #!!!!!!!!!
for i in g:
    if i:
        i = i.split(',')
        gg += i
gg = list(set(gg))
for i, j in enumerate(gg):
    ggg.append((j, i))
ggg = dict(ggg)
ggg

In [ ]:
titles_genres_dict = {}
for title in data_listed[0:50]:
    if title[8]:
        genres = title[8].split(',')
        temp = []
        for j in genres:
            temp.append((title[0], ggg[j]))
        titles_genres_dict[title[0]] = temp

In [ ]:
titles_genres_dict

In [ ]:
links_titles = []
for d in data_listed:
    links_titles.append((d[0], d[9]))

In [ ]:
links_titles #!!!!!!!!

In [ ]:
# insert_query = """
#     INSERT INTO genres (genre_id, genre_name)
#     VALUES (%s, %s)
#     """
# connection = create_connection(host_name, user_name, user_password, db_name, port)
# cursor = connection.cursor()
# try:
#     cursor.executemany(insert_query, ggg)
#     connection.commit()
#     print(f"{cursor.rowcount} rows inserted successfully")
# except Error as e:
#     print(f"The error '{e}' occurred")
# finally:
#     cursor.close()

In [57]:
titles_genres_dict

{'tt29515719': [('tt29515719', 11)],
 'tt29101369': [('tt29101369', 1)],
 'tt27301485': [('tt27301485', 9)],
 'tt29578104': [('tt29578104', 11)],
 'tt23874192': [('tt23874192', 16), ('tt23874192', 13), ('tt23874192', 8)],
 'tt29515862': [('tt29515862', 20)],
 'tt28045727': [('tt28045727', 11)],
 'tt27802756': [('tt27802756', 17), ('tt27802756', 11)],
 'tt22007328': [('tt22007328', 11)],
 'tt27301484': [('tt27301484', 1), ('tt27301484', 27)],
 'tt27898371': [('tt27898371', 6)],
 'tt22006694': [('tt22006694', 1)],
 'tt5892642': [('tt5892642', 11)],
 'tt27636322': [('tt27636322', 1)],
 'tt18069838': [('tt18069838', 13), ('tt18069838', 11), ('tt18069838', 19)],
 'tt27899117': [('tt27899117', 17), ('tt27899117', 2), ('tt27899117', 11)],
 'tt23214770': [('tt23214770', 23)],
 'tt22006416': [('tt22006416', 2), ('tt22006416', 1)],
 'tt25602582': [('tt25602582', 1)],
 'tt18546730': [('tt18546730', 13), ('tt18546730', 1), ('tt18546730', 23)],
 'tt22088562': [('tt22088562', 20)],
 'tt11097240': [(

In [ ]:
## !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! MAIN PART !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [58]:
connection = create_connection(host_name, user_name, user_password, db_name, port)
cursor = connection.cursor()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
for d in range(len(data_listed)):
    if d % 1000 == 0:
        cursor.close()
        connection.close() 
        connection = create_connection(host_name, user_name, user_password, db_name, port)
        cursor = connection.cursor()
    try:
        # Insert into titles
        insert_query_titles = """
        INSERT IGNORE INTO titles (tconst, title_type, title_name, description, year_start, year_end, is_adult)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query_titles, [data_listed[d][0], data_listed[d][1], data_listed[d][2], data_listed[d][10], data_listed[d][5], data_listed[d][6], data_listed[d][4]])
        connection.commit()
        print(f"Film {d} successfully inserted into table TITLES")
        
        # Insert into titles_image
        insert_query_links = """
            INSERT IGNORE INTO titles_image (tconst, image_url)
            VALUES (%s, %s)
            """
        cursor.execute(insert_query_links, [data_listed[d][0], data_listed[d][9]])
        print(f"Film {d}'s link for picture successfully inserted into table TITLES_IMAGE")

        insert_query_names_titles = """
    INSERT INTO names_titles (nconst, tconst, n_role, character_played, actor_importance)
    VALUES (%s, %s, %s, %s, %s)
        """
        names_titles = names_titles_data[data_listed[d][0]]
        try:
            if names_titles:
                cursor.executemany(insert_query_names_titles, names_titles)
                connection.commit()
                print(f"{cursor.rowcount} names inserted successfully")
            else:
                print('no names for this film')
        except Error as e:
            print(f"The error '{e}' occurred")
            
        insert_query = """
    INSERT INTO titles_genres (tconst, genre_id)
    VALUES (%s, %s)
    """
        titles_genres = titles_genres_dict[data_listed[d][0]]
        if titles_genres:
            try:
                cursor.executemany(insert_query, titles_genres)
                connection.commit()
                print(f"{cursor.rowcount} genres inserted successfully")
            except Error as e:
                print(f"The error '{e}' occurred")
        else:
            print('no genres for this film')
        try:
            if data_listed[d][10]:
                i = data_listed[d][10].lower()
                i = re.sub(r'[^\w\s]', '', i)
                i = re.sub(r'\d', '', i)
                tokens = word_tokenize(i)
                lemmas = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 2]
                i = ' '.join(lemmas)
                print(f'Film {d} lemmatized')
            else:
                i = None
                print(f'Film {d} has no description, lemmatization skipped')

            insert_query_descs = """
            INSERT IGNORE INTO description_modified (tconst, description)
            VALUES (%s, %s)
            """
            cursor.execute(insert_query_descs, (data_listed[d][0], i))
            connection.commit()
            print(f'Lemmatized description of film {d} has been inserted into DESCRIPTION_MODIFIED')

        except AttributeError:
            print(f'Film {d} has no description, lemmatization skipped')
            insert_query_descs = """
            INSERT IGNORE INTO description_modified (tconst, description)
            VALUES (%s, %s)
            """
            cursor.execute(insert_query_descs, (data_listed[d][0], None))
            connection.commit()
            print(f'NoneType description of film {d} has been inserted into DESCRIPTION_MODIFIED')

    except Exception as e:
        print(f'ERROR: {e}\nFilm {d} skipped')

    print('--------')

Connection to MySQL DB successful
Connection to MySQL DB successful
Film 0 successfully inserted into table TITLES
Film 0's link for picture successfully inserted into table TITLES_IMAGE
The error '1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`recsys_project_database`.`names_titles`, CONSTRAINT `names_titles_ibfk_1` FOREIGN KEY (`nconst`) REFERENCES `names` (`nconst`))' occurred
The error '1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`recsys_project_database`.`titles_genres`, CONSTRAINT `titles_genres_ibfk_2` FOREIGN KEY (`genre_id`) REFERENCES `genres` (`genre_id`))' occurred
Film 0 lemmatized
Lemmatized description of film 0 has been inserted into DESCRIPTION_MODIFIED
--------
Film 1 successfully inserted into table TITLES
Film 1's link for picture successfully inserted into table TITLES_IMAGE
The error '1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`recsys_project_database`.`names

KeyboardInterrupt: 

In [ ]:
query = "SELECT COUNT(*) AS total_rows FROM titles;"
connection = create_connection(host_name, user_name, user_password, db_name, port)
cursor = connection.cursor()
# Execute the query
cursor.execute(query)

# Fetch the result
result = cursor.fetchone()

# Print the result
print(f"Total rows in titles table: {result[0]}")

# Close the cursor and the connection
cursor.close()
connection.close()

In [ ]:
insert_query = """
    INSERT IGNORE INTO titles_image (tconst, image_url)
    VALUES (%s, %s)
    """
a = 0
cursor = connection.cursor()
for l in range(60051+1821+186+449, len(links_titles)):
    cursor.execute(insert_query, l)
    connection.commit()
    a += 1
    print(f'{a} rows inserted successfully')
# def rec(start, a):
#     try:
#         for i in range(start, len(links_titles), 500):
#             batch = links_titles[i:i + 500]
#             cursor.executemany(insert_query, batch)
#             connection.commit()
#             a += cursor.rowcount
#             if cursor.rowcount == 0:
#                 rec(start + a, a)
#             print(f"{cursor.rowcount} movies inserted successfully")
#             print(f"Total rows inserted: {a}")
#             time.sleep(0.1)
#     except Error as e:
#         print(f"The error '{e}' occurred")
# rec(60051+1821+186+449, a)
# Вставка данных порциями по 1000 записей
# try:
#     for i in range(60051+1821+186+449, len(links_titles), 500):
#         batch = links_titles[i:i + 500]
#         cursor.executemany(insert_query, batch)
#         connection.commit()
#         a += cursor.rowcount
#         print(f"{cursor.rowcount} movies inserted successfully")
#         print(f"Total rows inserted: {a}")
#         time.sleep(0.1)
# except Error as e:
#     print(f"The error '{e}' occurred")

# Закрытие курсора после завершения работы
cursor.close()
# 59050

In [ ]:
count_query = f"SELECT COUNT(*) FROM titles_image"
cursor = connection.cursor()
try:
    cursor.execute(count_query)
    row_count = cursor.fetchone()[0]
    print(f"Number of rows in 'titles_image' table: {row_count}")
except Error as e:
    print(f"The error '{e}' occurred")
finally:
    cursor.close()

In [ ]:
query = """
SELECT t.tconst, t.title_type, t.title_name, t.description AS original_description, 
       dm.description AS modified_description, t.year_start, t.year_end, t.is_adult
FROM titles t
LEFT JOIN description_modified dm ON t.tconst = dm.tconst
LIMIT 5;
"""
connection = create_connection(host_name, user_name, user_password, db_name, port)
cursor = connection.cursor()
# Execute the query
cursor.execute(query)

# Fetch the results
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

# Close the cursor and the connection
cursor.close()
connection.close()

In [ ]:
data_listed = titlebasics_cleaned.to_numpy().tolist()

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

c = 0
bad_films = []

# Assuming `connection` is your database connection object
cursor = connection.cursor()

for d in data_listed:
    try:
        # Insert into titles
        insert_query_titles = """
        INSERT IGNORE INTO titles (tconst, title_type, title_name, description, year_start, year_end, is_adult)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query_titles, [d[0], d[1], d[2], d[10], d[5], d[6], d[4]])
        connection.commit()
        print(f"Film {c} successfully inserted into table TITLES")

        # Process and insert description
        try:
            if d[10]:
                i = d[10].lower()
                i = re.sub(r'[^\w\s]', '', i)
                i = re.sub(r'\d', '', i)
                tokens = word_tokenize(i)
                lemmas = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 2]
                i = ' '.join(lemmas)
                print(f'Film {c} lemmatized')
            else:
                i = None
                print(f'Film {c} has no description, lemmatization skipped')

            insert_query_descs = """
            INSERT INTO description_modified (tconst, description)
            VALUES (%s, %s)
            """
            cursor.execute(insert_query_descs, (d[0], i))
            connection.commit()
            print(f'Lemmatized description of film {c} has been inserted into DESCRIPTION_MODIFIED')

        except AttributeError:
            print(f'Film {c} has no description, lemmatization skipped')
            insert_query_descs = """
            INSERT IGNORE INTO description_modified (tconst, description)
            VALUES (%s, %s)
            """
            cursor.execute(insert_query_descs, (d[0], None))
            connection.commit()
            print(f'NoneType description of film {c} has been inserted into DESCRIPTION_MODIFIED')

    except Exception as e:
        print(f'ERROR: {e}\nFilm {c} skipped')
        bad_films.append(d)

    print('--------')
    c += 1

# Closing the cursor and connection
cursor.close()
connection.close()

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
c = 0
bad_films = []
for d in data_listed:
    i = d[10]
    insert_query_titles = """
    INSERT IGNORE INTO titles (tconst, title_type, title_name, description, year_start, year_end, is_adult)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    cursor = connection.cursor()
    try:
        cursor.execute(insert_query_titles, [d[0], d[1], d[2], d[10], d[5], d[6], d[4]])
        connection.commit()
        print(f"film {c} successfilly inserted into table TITLES")
        try:
            i = i.lower()
            i = re.sub(r'[^\w\s]', '', i)
            i = re.sub(r'\d', '', i)
            tokens = word_tokenize(i)
            lemmas = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 2]
            i = [' '.join(lemmas)]
            print(f'film {c} lemmatized')
            insert_query_descs = """
    INSERT INTO description_modified (tconst, description)
    VALUES (%s, %s)
    """
            cursor.execute(insert_query_descs, (d[0], i))
            connection.commit()
            print(f'lemmatized description of film {c} has inserted to DESCRIPTION_MODDED')
        except AttributeError:
            i = None
            print(f'film {c} has no description, lemmatization skipped')
            insert_query_descs = """
    INSERT IGNORE INTO description_modified (tconst, description)
    VALUES (%s, %s)
    """
            cursor.execute(insert_query_descs, (d[0], i))
            connection.commit()
            print(f'NONETYPE of film {c} has inserted to DESCRIPTION_MODDED')
    except Exception as e:
        print(f'ERROR: {e}\nFilm {c} skipped')
        bad_films.append(d)
    print('--------')
    c += 1

In [ ]:
# movies = titlebasics[['tconst', 'titleType', 'primaryTitle', 'description','startYear', 'endYear']].to_numpy().tolist()

In [ ]:
titlebasics['genres'].unique()

In [ ]:
# done
all_genres = []
for g in titlebasics['genres'].unique()[1:]:
    all_genres.extend(g.split(','))

all_genres = list(set(all_genres))
# all_genres.remove('\\N')
all_genres = [(index + 1, genre) for index, genre in enumerate(all_genres)]
all_genres

In [ ]:
# done
all_genres_dict = dict(all_genres)

In [ ]:
batch_size = 500
insert_query = """
    INSERT IGNORE INTO titles (tconst, title_type, title_name, description, year_start, year_end)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
cursor = connection.cursor()
c = 0
broken_movies = []
for movie in range(0, len(movies)):
    try:
        for i in range(0, len(movies), batch_size):
            batch = movies[i:i + batch_size]
            cursor.executemany(insert_query, batch)
            connection.commit()
            c += cursor.rowcount
            print(f"{cursor.rowcount} movies inserted successfully\n'total rows insserted: {c}'")
    except Error as e:
        print(f"The error '{e}' occurred")
        broken_movies.extend(movies[i:i + batch_size])

In [ ]:
broken_movies_copy = broken_movies

In [ ]:
broken_movies_copy

In [ ]:
titlebasics

In [ ]:
desc_modded = titlebasics[['tconst', 'desc_modded']].to_numpy().tolist()

In [ ]:
desc_modded

In [ ]:
insert_query = """
    INSERT INTO description_modified (tconst, description)
    VALUES (%s, %s)
    """
batch_size = 500
cursor = connection.cursor()
for i in range(0, len(desc_modded), batch_size):
    batch = desc_modded[i:i + batch_size]
    try:
        cursor.executemany(insert_query, batch)
        connection.commit()
        print(f"{len(batch)} records inserted successfully")
    except Error as e:
        print(f"The error '{e}' occurred during batch insertion")
        connection.rollback()

In [ ]:
count_query = "SELECT COUNT(*) FROM titles_image"
cursor = connection.cursor()
try:
    cursor.execute(count_query)
    row_count = cursor.fetchone()[0]
    print(f"Number of rows in 'titles' table: {row_count}")
except Error as e:
    print(f"The error '{e}' occurred")

In [ ]:
# done
insert_query = """
    INSERT INTO genres (genre_id, genre_name)
    VALUES (%s, %s)
    """
try:
    cursor = connection.cursor()
    cursor.executemany(insert_query, genres)
    connection.commit()
    print(f"{cursor.rowcount} records inserted successfully into genres table")
except Error as e:
    print(f"The error '{e}' occurred")

In [ ]:
titlebasics = pd.read_csv(r'C:\Users\mayko\YandexDisk-makorablina@edu.hse.ru\CodingProjects\recsys_project\data\title.basics.tsv', sep = '\t')
titlebasics = pd.merge(titlebasics, desc_df, on='tconst', how='inner')

In [ ]:
titlebasics 

In [ ]:
all_genres = []
for g in titlebasics['genres'].unique():
    all_genres.extend(g.split(','))

all_genres = list(set(all_genres))
all_genres.remove('\\N')
all_genres = [(index + 1, genre) for index, genre in enumerate(all_genres)]
all_genres

In [ ]:
insert_query = """
    INSERT INTO names (nconst, name, birth_year, death_year)
    VALUES (%s, %s, %s, %s)
    """
try:
    cursor = connection.cursor()
    cursor.executemany(insert_query, names)
    connection.commit()
    print(f"{cursor.rowcount} records inserted successfully into names table")
except Error as e:
    print(f"The error '{e}' occurred")

In [ ]:
insert_query = """
    INSERT INTO names_titles (nconst, tconst)
    VALUES (%s, %s)
    """
try:
    cursor = connection.cursor()
    cursor.executemany(insert_query, names_titles)
    connection.commit()
    print(f"{cursor.rowcount} records inserted successfully into names_titles table")
except Error as e:
    print(f"The error '{e}' occurred")

In [ ]:
insert_query = """
    INSERT INTO ratings (tconst, rating, num_votes)
    VALUES (%s, %s, %s)
    """
try:
    cursor = connection.cursor()
    cursor.executemany(insert_query, ratings)
    connection.commit()
    print(f"{cursor.rowcount} records inserted successfully into ratings table")
except Error as e:
    print(f"The error '{e}' occurred")

In [ ]:
insert_query = """
    INSERT INTO titles_image (tconst, image_url)
    VALUES (%s, %s)
    """
try:
    cursor = connection.cursor()
    cursor.executemany(insert_query, titles_images)
    connection.commit()
    print(f"{cursor.rowcount} records inserted successfully into titles_image table")
except Error as e:
    print(f"The error '{e}' occurred")